In [1]:
from hsflfm.analysis import (
    ResultManager,
    ResultPlotter,
    BulkAnalyzer,
    convert_to_percentile,
    get_random_percentile_index,
    sort_by_camera,
    get_percentiles
)

import os
from matplotlib import pyplot as plt
import torch
from hsflfm.util import load_dictionary, save_dictionary, play_video
from pathlib import Path
import numpy as np

In [2]:
# get all the filenames
all_filenames = []
f = "temporary_result_storage_2"
folders = os.listdir(f)
for inner in folders:
    path = Path(f) / inner
    if path.is_dir():
        filenames = os.listdir(path)
        for filename in filenames:
            if filename[-4:] == "json":
                all_filenames.append(str(path / filename))

In [3]:
analyzer = BulkAnalyzer(all_filenames)

In [4]:
reload = True
if reload:
    analyzer.load_results()
    save_dictionary(analyzer.all_results, "temp_loaded_results_2.json")
else:

    analyzer.all_results = load_dictionary("temp_loaded_results_2.json")

    for key, value in analyzer.all_results.items():

        if key == "specimen_number":

            analyzer.all_results[key] = np.asarray(value)

            continue

        analyzer.all_results[key] = torch.asarray(value)

 11%|█         | 36/322 [00:08<01:04,  4.45it/s]


KeyboardInterrupt: 

In [ ]:
%debug

In [ ]:
# look at all results on the mesh

p = analyzer.all_results["mesh_points"]

# jitter the points

jitter = 100

rand = (torch.rand(p.shape) - 0.5) * jitter

p = p + rand

v = analyzer.all_results["normalized_displacement"][:, 2]

v = convert_to_percentile(v)

ResultPlotter.plot_mesh_with_points(
    points=p,
    opacity=0.1,
    point_values=v,
    points_on_surface=False,
    marker_dict={"size": 2, "colorscale": "Turbo"},
)

In [ ]:
# histogram flow differences
key = "average_flow_error"
all_flow = torch.mean(torch.abs(analyzer.all_results[key]), axis=1)
all_flow, _ = torch.sort(all_flow)

# cut-off at some percentile
cutoff = 0.995
cutoff_index = int(len(all_flow) * cutoff)

bins = plt.hist(all_flow[:cutoff_index], bins=50, alpha=0.5, label="all 3 cameras")
width = bins[1][1] - bins[1][0]

# add in the top two
flow = analyzer.get_top_values(key)
flow, _ = torch.sort(torch.mean(torch.abs(flow), axis=1))
flow = flow[:cutoff_index]
bins = np.arange(min(flow), max(flow) + width, width)
_ = plt.hist(flow, bins=bins, alpha=0.5, label="top 2 cameras")

plt.legend()
plt.ylabel("Point Count")
plt.xlabel("Flow Error (pixels)")
plt.title("Flow error in region around strike")

In [ ]:
flow.shape

In [ ]:
# histogram flow differences
key = "average_flow_sq"
all_flow = torch.mean(torch.abs(analyzer.all_results[key]), axis=1)
all_flow, _ = torch.sort(all_flow)

# cut-off at some percentile
cutoff = 0.95
cutoff_index = int(len(all_flow) * cutoff)

bins = plt.hist(all_flow[:cutoff_index], bins=50, alpha=0.5, label="all 3 cameras")
width = bins[1][1] - bins[1][0]

# # add in the top two
flow = analyzer.get_top_values(key)
flow, _ = torch.sort(torch.mean(torch.abs(flow), axis=1))
# cut-off at some percentile
cutoff = 0.99
cutoff_index = int(len(all_flow) * cutoff)
flow = flow[:cutoff_index]
bins = np.arange(min(flow), max(flow) + width, width)
_ = plt.hist(flow, bins=bins, alpha=0.5, label="top 2 cameras")

plt.legend()
plt.ylabel("Point Count")
plt.xlabel("Flow Error (pixels)")
plt.title("Flow error in region around strike")

In [ ]:
# histogram flow differences
key = "average_huber_loss"
all_flow = torch.mean(torch.abs(analyzer.all_results[key]), axis=1)
all_flow, _ = torch.sort(all_flow)

# cut-off at some percentile
cutoff = 0.995
cutoff_index = int(len(all_flow) * cutoff)

bins = plt.hist(all_flow[:cutoff_index], bins=50, alpha=0.5, label="all 3 cameras")
width = bins[1][1] - bins[1][0]

# add in the top two
flow = analyzer.get_top_values(key)
flow, _ = torch.sort(torch.mean(torch.abs(flow), axis=1))
flow = flow[:cutoff_index]
bins = np.arange(min(flow), max(flow) + width, width)
_ = plt.hist(flow, bins=bins, alpha=0.5, label="top 2 cameras")

plt.legend()
plt.ylabel("Point Count")
plt.xlabel("Huber Loss")
plt.title("Huber Loss in region around strike")

In [ ]:
# percentile differences between huber and flow in top 2 cameras
p0 = analyzer.get_percentile("average_flow_error", num_cams=2)
p1 = analyzer.get_percentile("average_flow_sq", num_cams=2)

diffs = torch.abs(p1 - p0)

_ = plt.hist(diffs, bins=50)

In [ ]:
# percentile differences between huber and flow in top 2 cameras
huber_percentiles = analyzer.get_percentile("average_huber_loss", num_cams=2)
flow_percentiles = analyzer.get_percentile("average_flow_error", num_cams=2)

diffs = torch.abs(huber_percentiles - flow_percentiles)

_ = plt.hist(diffs, bins=50)

In [ ]:
# look at some points with bad flow
array = torch.mean(analyzer.get_top_values("average_flow_sq", num_cams=2), axis=1)
index = get_random_percentile_index(array.numpy(), 90, 100)

specimen_number = analyzer.all_results["specimen_number"][index]
point_number = int(analyzer.all_results["point_number"][index])
strike_number = int(analyzer.all_results["strike_number"][index])

print(specimen_number, "point", point_number, "strike", strike_number)
print("flow error: {:.5f} pixels".format(array[index]))
print("percentile: {:.0f}%".format(100 * get_percentiles(array.numpy(), float(array[index]))))

In [ ]:
0.025**2

In [ ]:
# specimen_number = "20240507_OB_2"
# point_number = 31
# strike_number = 10

In [ ]:
filename = (
    f"temporary_result_storage_2/{specimen_number}/strike_{strike_number}_results.json"
)
assert os.path.exists(filename)

result_info = load_dictionary(filename)
plotter = ResultPlotter(result_info)

plotter.result_info["removed_points"]

In [ ]:
flow_diffs = plotter.result_manager.flow_diff_around_strike()
_, sorted = sort_by_camera(flow_diffs[:, :, None], treat_individually=False)
values = sorted.squeeze()[:, :2]
values = torch.mean(values, axis=1)
_ = plotter.scatter_values(values, highlight_point=point_number)

In [ ]:
plotter.scatter_peak_disp(highlight_point=point_number, cmap="turbo")
plotter.scatter_peak_disp(highlight_point=point_number, cmap="turbo", with_image=True)

In [ ]:
_ = plotter.plot_camera_weight(point_number)

In [ ]:
_ = plotter.plot_displacement(point_number)

In [ ]:
plotter.show_flow_differences(point_number)

In [ ]:
_ = plotter.plot_all_displacement(highlight_point=point_number)

In [ ]:
vid = plotter.make_point_track_video(highlight_point=point_number)

In [ ]:
play_video(vid)

In [ ]:
vid = plotter.get_arrow_video()

In [ ]:
play_video(vid)

In [ ]:
m = plotter.result_manager.point_mesh_locations
ResultPlotter.plot_mesh_with_points(points=m)

In [ ]:
# look into more:
# 20240507_OB_2 point 31 strike 10
# 20240502_OB_6 29
# 20240502_OB_2 alignment is super off 

# good examples
# 20240418_OB_1 barely any movement but clear pattern

In [ ]:
# suggested threshold for being used in global movement calculation:
# 0.025 average error with top two cameras
# in region surrounding peak 
# or... maybe squared error of 0.0015

In [ ]:
cam = 0 
pn = np.asarray(result_info["point_numbers"], dtype=int)
p1 = np.asarray(result_info["strike1_match_points"][cam])[pn, :2]
p2 = np.asarray(result_info["match_points"][cam])
diff = np.linalg.norm(p1 - p2, axis=1)

_ = plotter.scatter_values(diff)